In [4]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
    
# funciones desarrolladas
from functions.cargar_data import *
from functions.modelos import *

In [5]:
# carga datos diádicos
dd_deptos = cargar_dd_deptos()
print(dd_deptos.columns)
dd_deptos.head(2)

Index(['cod', 'depto_origen', 'depto_destino', 'personas_mig',
       'personas_mig_18_25', 'personas_mig_26_35', 'personas_mig_100k',
       'pob_origen', 'pob_destino', 'dist', 'dist_km', 'pbi_origen',
       'pbi_porcen_ori', 'pbi_destino', 'pbi_porcen_des', 'largo_limite',
       'dummy_limit', 'empresas_origen', 'empresas_destino', 'log_pbi_destino',
       'log_dist', 'nom_depto_orig', 'nom_depto_des', 'largo_limite_km'],
      dtype='object')


,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,pbi_porcen_des,largo_limite,dummy_limit,empresas_origen,empresas_destino,log_pbi_destino,log_dist,nom_depto_orig,nom_depto_des,largo_limite_km
0,102,1,2,914,114,247,891,1318755,73377,610726,...,1.5,0.0001,False,3124,55,16.469807,13.322404,MONTEVIDEO,ARTIGAS,1.000000e-07
1,103,1,3,33127,3757,7904,0,1318755,520173,22544,...,9.3,60107.0000,True,3124,563,18.273415,10.023224,MONTEVIDEO,CANELONES,6.010700e+01


## Modelos con Montevideo

### 1. Modelo 1: dummy limit, largo límite, PBI destino y distancia

In [6]:
# define el string tomado como fórmula
formula = """
personas_mig ~ nom_depto_orig + dummy_limit
+ largo_limite_km + pbi_destino_millardos + dist_km + -1
"""

# entrena y ajusta el modelo
family = sm.families.Poisson()
prodSim = smf.glm(formula=formula, data=dd_deptos, family = family).fit()

# resultados
prodSim.summary()

PatsyError: Error evaluating factor: NameError: name 'pbi_destino_millardos' is not defined
    personas_mig ~ nom_depto_orig + dummy_limit + largo_limite_km + pbi_destino_millardos + dist_km + -1
                                                                    ^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print_scores_simple(dd_deptos.personas_mig, prodSim.predict())

In [ ]:
fig = plt.figure()
sns.regplot(x=predicted_counts, y=actual_counts, line_kws={"color": "green"})

fig.suptitle('Groud truth vs. estimaciones del modelo')
plt.xlabel('Valores estimados')
plt.ylabel('Groud truth')

In [ ]:
prodSim.save("modelos/restringido_origen_mvo.pickle")

## Modelo sin Montevideo

In [ ]:
dd_deptos_no_mvo = dd_deptos.loc[(dd_deptos.depto_origen!=1) & (dd_deptos.depto_destino!=1)]

In [ ]:
# define el string tomado como fórmula
formula = """
personas_mig ~ nom_depto_orig + dummy_limit
+ largo_limite_km + pbi_destino_millardos + dist_km -1
"""

# entrena y ajusta el modelo
family = sm.families.Poisson()
prodSim = smf.glm(formula=formula, data=dd_deptos_no_mvo, family = family).fit()

# resultados
prodSim.summary()

In [ ]:
print_scores_simple(dd_deptos_no_mvo.personas_mig, prodSim.predict())

In [ ]:
prodSim.save("modelos/restringido_origen_noMVO.pickle")